In [1]:
from torch.utils.data import Dataset, DataLoader, Sampler
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageTk
import cv2
import os
import random
import csv
from skimage.util import img_as_ubyte
import tkinter
import time, datetime
import pandas as pd

In [2]:
def normalize(array):
    return (array-np.min(array))/(np.max(array)-np.min(array))

In [3]:
def multiply(img,mask):
    img_new = img.copy()
    img_new[:,:,0] = np.multiply(img_new[:,:,0],mask)
    img_new[:,:,1] = np.multiply(img_new[:,:,1],mask)
    img_new[:,:,2] = np.multiply(img_new[:,:,2],mask)
    return img_new

In [4]:
def get_filelist(root,save_path):
    filename_list = []

    for emotion in range(8):
        #get list of filenames
        one_folder = os.listdir(root+str(emotion))
        
        if save_path is not None:
            #create label lists
            true = [emotion]*len(one_folder)
            temp = list(range(8))*5
            false = [i for i in temp if i is not emotion]   #this means that for a specific image, its false label will always be the same.
        else:
            true, false = [emotion], [random.choice([i for i in range(8) if i is not emotion])]
        
        #assign true/false labels to each filename
        for i, filename in enumerate(one_folder):
            chunk = [str(emotion)+'/'+filename,true[i],false[i]]
            filename_list.append(chunk)
        
    random.shuffle(filename_list)
    
    return filename_list

In [5]:
def change_filename(filename):
    if filename[1] is '/':
        emotion = filename.split('/')[0]
        picnum = filename.split('/')[1].split('.')[0]
    else:
        emotion = filename.split('.')[0]
        picnum = ''
    return emotion+'_'+picnum

In [6]:
img = cv2.imread('trial_samples/1/25.jpg')
blurred = blur(img)
cv2.imwrite('sample.jpg',blurred)

NameError: name 'blur' is not defined

In [ ]:
def blur(img,k=70):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = np.stack((gray,)*3, axis=-1)
    blurred = cv2.blur(gray,(k,k))
    return blurred
def read_blur(root,filename,k=70):
    cvimg = cv2.imread(root+filename)
    cvimg = cv2.resize(cvimg,(256,256))
    gray = cv2.cvtColor(cvimg, cv2.COLOR_BGR2GRAY)
    gray = np.stack((gray,)*3, axis=-1)
    blurred = cv2.blur(gray,(k,k))
    im_pil = Image.fromarray(blurred)
    return im_pil

In [7]:
def circle_mask(click,r=15):
    x = np.linspace(0, 255, 256)
    y = np.linspace(0, 255, 256)
    x, y = np.meshgrid(x, y)
    mask = np.sqrt((x-click[0])**2+(y-click[1])**2)

    for x in range(256):
        for y in range(256):
            if mask[x,y] < r:
                    mask[x,y] = 1
            elif mask[x,y] >= r:
                    mask[x,y] = 0
    return mask

In [8]:
def reveal(root,filename,mask):
    img = cv2.imread(root+filename)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    blurred = blur(img)
    delete_mask = -(mask-1)
    deleted = multiply(blurred,delete_mask)
    revealed = multiply(img_rgb,mask)
    added = np.add(deleted,revealed)
    return added

In [9]:
import os
import tkinter as tk
from PIL import Image
from PIL import ImageTk
import random
from random import choice

class MyFrame:
    
    def __init__(self, parent, root_folder, save_path, filelist):

        my_frame = tk.Frame(parent)
        my_frame.pack()
        
        self.root_folder = root_folder
        self.save_path = save_path
        self.filelist = filelist
        
        self.instruction = tk.Label(text='자세히 보고 싶은 부분을 클릭하세요.\n이 사람은 어떤 표정을 짓고 있나요?')
        self.instruction.pack()
        
        self.button1 = tk.Button(text='Start')
        self.button1.place(x=50,y=350,width=100)
        self.button1.bind('<ButtonRelease-1>', self.button_clicked)
        
        self.button2 = tk.Button(text='Start')
        self.button2.place(x=250,y=350,width=100)
        self.button2.bind('<ButtonRelease-1>', self.button_clicked)
        
        #get start image
        start_img = Image.open('35samples/start.jpg')   #image doesn't show...
        start_img = ImageTk.PhotoImage(start_img)

        #set label for displaying image
        self.label = tk.Label(image=start_img)
        self.label.pack()
        self.label.bind('<ButtonRelease-1>',self.image_clicked)
        
        #set initial variables
        self.count = 0 
        self.start_time = 0
        self.filename = 'start.jpg'
        self.true, self.false, self.count = 111, 111, 0
        self.mask = np.zeros((256,256))
        self.added = cv2.imread('35samples/start.jpg')
        
    def button_clicked(self, event):
                      
        if self.save_path is not None:  #save mask, masked, label, choice and time
            
            fname = change_filename(self.filename)
            cv2.imwrite(self.save_path+'/'+fname+'_mask.jpg',(self.mask*255).astype(int))
            cv2.imwrite(self.save_path+'/'+fname+'_masked.jpg', cv2.cvtColor(self.added, cv2.COLOR_RGB2BGR))
            self.mask = np.zeros((256,256))   #initialize mask and masked
            self.added = cv2.imread('35samples/start.jpg')
            
            chosen = event.widget.cget("text")
            fieldnames = ['filename','true','false','chosen','correct','time','count']            
            if os.path.isfile(self.save_path+'/choices.csv'):
                f = open(self.save_path+'/choices.csv','a', newline='')
                wr = csv.DictWriter(f,fieldnames=fieldnames)
            else:
                f = open(self.save_path+'/choices.csv','w', newline='')
                wr = csv.DictWriter(f,fieldnames=fieldnames)
                wr.writeheader()
            if self.filename[1] == '/':
                wr.writerow({'filename':self.filename,'true':self.true,'false':self.false,'chosen':emo_to_num[chosen],
                             'correct':self.true==emo_to_num[chosen],'time':round(time.time()-self.start_time,3),'count':self.count})
            f.close()
              
        #get filename from list
        try:
            file = next(self.filelist)
            self.filename = file[0]
            self.true, self.false = file[1], file[2]
        except StopIteration:
            self.filename = 'end.jpg'
            self.true, self.false = 112, 112
        
        if self.filename is not 'end.jpg':
            blurred = read_blur(self.root_folder,self.filename)
            img = ImageTk.PhotoImage(blurred)
        else:
            img = Image.open(self.root_folder+self.filename)
            img = ImageTk.PhotoImage(img)
            
        self.label.config(image=img)
        self.label.image = img
        
        #randomly assign true, false to btn1, btn2
        rand_list = [emotion_labels[self.true],emotion_labels[self.false]]
        random.shuffle(rand_list)
        
        self.button1['text'] = rand_list[0]
        self.button2['text'] = rand_list[1]
        
        self.count = 0  #reset number of clicks
      
    def image_clicked(self, event):
        
        #mask and show image
        if self.count == 0:
            self.mask = circle_mask([event.x,event.y])
            self.start_time = time.time()
        else:
            self.mask = self.mask + circle_mask([event.x,event.y])
            self.mask = np.clip(self.mask,0,1)

        self.added = reveal(self.root_folder,self.filename,self.mask)
        img = ImageTk.PhotoImage(Image.fromarray(self.added))
        self.label.config(image=img)
        self.label.image = img   
        
        self.count = self.count + 1        
        
        if self.save_path is not None:
            #save click position and timestamp to csv
            elapsed = time.time() - self.start_time
            fieldnames = ['filename','x_pos','y_pos','time']            
            if os.path.isfile(self.save_path+'/clicks.csv'):
                f = open(self.save_path+'/clicks.csv','a', newline='')
                wr = csv.DictWriter(f,fieldnames=fieldnames)
            else:
                f = open(self.save_path+'/clicks.csv','w', newline='')
                wr = csv.DictWriter(f,fieldnames=fieldnames)
                wr.writeheader()
            wr.writerow({'filename':self.filename,'x_pos':event.x,'y_pos':event.y,'time':round(elapsed,3)}) 
            f.close()



In [10]:
def sess_end(num):
    between_session = tk.Tk()
    between_session.title('Between session')
    between_session.geometry('400x400+600+300')
    if num is not 4:
        label = tk.Label(text='{}번째 세션이 끝났습니다. 잠시 휴식을 취해주세요.\n 이 창을 끄면 다음 세션이 시작됩니다.'.format(num))
    else:
        label = tk.Label(text='실험이 모두 끝났습니다.\n 수고하셨습니다.')
    label.place(relx=0.5, rely=0.5, anchor="center")

    between_session.mainloop()

In [11]:
def session(root_folder,filelist,save_path):
       
    #create instance
    experiment = tk.Tk()
    experiment.title('Experiment')
    experiment.geometry('400x400+600+300')
    frame = MyFrame(experiment,root_folder,save_path,filelist)

    experiment.mainloop()

In [15]:
#root folder
trial_root = '../trial_samples/'
main_root = '../35samples/'

emotion_labels_eng = {0: 'Neutral', 1: 'Happy', 2: 'Sad', 3: 'Surprise', 4: 'Fear', 5: 'Disgust', 6: 'Anger', 7: 'Contempt', 111: 'Start', 112: 'End'}
emotion_labels = {0: '중립', 1: '행복', 2: '슬픔', 3: '놀람', 4: '공포', 5: '혐오', 6: '분노', 7: '경멸', 111:'Start', 112: 'End'}
emo_to_num = {v: k for k, v in emotion_labels.items()}

def experiment():
    print(datetime.datetime.now())  
    
    date = input('Enter date (yymmdd):')
    participant_num = input('Enter participant number:')
    save_path = '../experiment_data/'+date+'_'+participant_num
    os.mkdir(save_path)
    
    trial_list = iter(get_filelist(trial_root,None))   #prepare filename list
    filelist_whole = get_filelist(main_root,save_path)
    quarter = int(len(filelist_whole)/4)
    filelist1 = iter(filelist_whole[:quarter])   #divide filelist into four sessions
    filelist2 = iter(filelist_whole[quarter:quarter*2])
    filelist3 = iter(filelist_whole[quarter*2:quarter*3])
    filelist4 = iter(filelist_whole[quarter*3:quarter*4])
    
    session(trial_root,trial_list,None)
    session(main_root,filelist1,save_path)
    sess_end(1)
    session(main_root,filelist2,save_path)
    sess_end(2)
    session(main_root,filelist3,save_path)
    sess_end(3)
    session(main_root,filelist4,save_path)
    sess_end(4)
    
    print(datetime.datetime.now())
    
    choices = pd.read_csv(save_path+'/choices.csv')
    print(choices['correct'].mean())
experiment()

2021-06-28 13:12:55.442342


Enter date (yymmdd): 0
Enter participant number: 1


IndexError: list index out of range